## Libraries

In [133]:
!pip install numpy
!pip install pandas
!pip install numpy
!pip install matplotlib

import matplotlib.pyplot as plt

import sklearn.metrics as sklm
import copy
import json

import pandas as pd
import numpy as np

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## Utils

In [134]:
PATH = "./"

In [135]:
def retrieve_data(flow_interval):
    path = "{0}dataset/dataset_flow_{1}.csv".format(PATH, flow_interval)
    print(PATH)
    data = pd.read_csv(path, ';')
    
    data['Flow'].apply(int)
    data['AveSpeed'].apply(float)
    data['Density'].apply(float)
    data['Sunday'].apply(int)
    data['Monday'].apply(int)
    data['Tuesday'].apply(int)
    data['Wednesday'].apply(int)
    data['Thursday'].apply(int)
    data['Friday'].apply(int)
    data['Saturday'].apply(int)
      
    return data

In [136]:
def print_json (obj):
  print(json.dumps(obj, sort_keys=True, indent=4))

In [137]:
def store(obj, path, name):
  with open("{0}{1}/{2}.json".format(PATH, path, name), 'w') as json_file:
    json.dump(obj, json_file, sort_keys=True, indent=4)

## Testing

In [138]:
# Model Parameters
FLOW_INTERVAL = 150 # the interval size for each flow
ALPHA = 0.8

In [139]:
def simple_exponential_smoothing(X):
  global result_data
  
  name = "SES"
  expected, observed = [], []

  X_test = np.array_split(X, 10)
  F = X_test[8][-1]
  for x in X_test[9]:
    F1 = ALPHA * x + (1 - ALPHA) * F
    
    expected.append(int(x))
    observed.append(float(F))
    F = F1
  
  result_data['results']['RMSE'] = sklm.mean_squared_error(expected, observed, squared=False)
  result_data['results']['expected'] = expected
  result_data['results']['observed'] = observed
  store(result_data, "results", "{0}_alpha_{1}".format(name, ALPHA))

In [140]:
global result_data
  
result_data = {
    'results': {},
    'meta': {
      'FLOW_INTERVAL': FLOW_INTERVAL,
    }
}

data = retrieve_data(FLOW_INTERVAL)
sequence = np.array(data['Flow'])
simple_exponential_smoothing(sequence)

./


In [141]:
result_data['results']['RMSE']

8.604186284879553

In [142]:
def plot_prediction (Y, Y_hat, title):
  """ Plot Prediction
  
  Plot the prediction (Flow x Time) of what was expected and what
  was predicted.
  """
  name = f"{title} (alpha {ALPHA})"
  path = f"plots/prediction/{name}"
    
  plt.plot(Y)
  plt.plot(Y_hat)
  plt.title(f"Predição do Modelo {title}")
  plt.ylabel('Fluxo')
  plt.xlabel('Tempo')
  plt.legend(['esperado', 'observado'], loc='upper left')
  plt.rcdefaults()

  plt.savefig(path + ".png", bbox_inches='tight')
  # plt.savefig(path + ".pdf")

  plt.close('all')

In [143]:
expected = result_data['results']['expected']
observed = result_data['results']['observed']
plot_prediction(expected, observed, "SES")